In [1]:
#Topic Mining

In [2]:
import warnings

In [3]:
warnings.filterwarnings('ignore')

In [4]:
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig

In [5]:
import pandas as pd
import spacy
import time
import nltk

C:\Users\asad_\Anaconda3\lib\site-packages\thinc\neural\train.py:7: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from .optimizers import Adam, linear_decay
C:\Users\asad_\Anaconda3\lib\site-packages\thinc\check.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, Sized, Iterable, Callable
C:\Users\asad_\Anaconda3\lib\site-packages\thinc\check.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, Sized, Iterable, Callable
C:\Users\asad_\Anaconda3\lib\site-packages\nltk\decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, 

In [10]:
# Only Arizona Businesses, Change if needed
restaurant_file='processed_data/cleaned_restaurants.csv'
reviews_file   ='processed_data/cleaned_reviews.csv'

# Number of topic
NUM_TOPICS = 5

In [11]:
%%time
# This is the large Spacy English Library
nlp = spacy.load('en_core_web_lg')

#  Stats
## Wall time: 12.6 s

Wall time: 12.6 s


In [12]:
%%time
# Read Businesses
all_restaurants = pd.read_csv(restaurant_file).drop(labels='Unnamed: 0', axis=1)

#  Stats
## Wall time: 68 ms

Wall time: 68 ms


In [14]:
%%time
# Read all reviews
all_reviews = pd.read_csv(reviews_file).drop(labels='Unnamed: 0', axis=1)

#  Stats
## Wall time: 55.6 s

Wall time: 55.6 s


In [15]:
%%time
# Top 5 Reviews
all_reviews.head()

Wall time: 0 ns


,business_id,name,business_stars,review_count,review_id,review_stars,useful,text,topic_text,sentiment_text
0,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,2.5,128,6W0MQHmasK0IsaoDo4bmkw,3.0,3,My girlfriend and I went for dinner at Emerald...,girlfriend dinner thursday night workout arriv...,girlfriend dinner chinese. thursday night work...
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,2.5,128,BeeBfUxvzD4qNX4HxrgA5g,3.0,0,We've always been there on a Sunday so we were...,sunday saturday dim_sum luck surprise dish col...,sunday saturday dim_sum. busy. no luck surpris...
2,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,2.5,128,A1D2kUnZ0HTroFreAheNSg,3.0,0,"***No automatic doors, not baby friendly!*** I...",door post_partum dim_sum dish dinner time door...,no_automatic door not_baby friendly frequent c...
3,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,2.5,128,2pf45Stf-pNew-xgTababQ,1.0,1,"Horrible service,\nI went there tonight with m...",tonight boyfriend pass couple time want try fr...,horrible_service tonight boyfriend because pas...
4,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,2.5,128,RHhlmL07evgAdPaXQV8Omg,4.0,2,One of the gauges of a good Chinese restaurant...,gauge patronize patron wife dim_sum brunch wee...,gauge good chinese_number chinese_people patro...


In [17]:
len(all_reviews)

3527902

In [36]:
restaurant = all_restaurants.iloc[[0]].business_id.values[0]

In [37]:
restaurant

'QXAEGFB4oINsVuTFxEYKFQ'

In [89]:
reviews = all_reviews.query(' business_id == "'+'44YFU284Z3KDEy25QyVoUw'+'" ')['topic_text']

In [90]:
reviews = [u''+str(txt).replace('..','.').replace('  ',' ') for txt in reviews]

In [91]:
reviews[1]

'rate walk soup p.m. destroy need clear_sinuse trick tell supper time bowl soup home look cup soup price soup'

In [92]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures

In [93]:
# Creating a vectorizer
vectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(reviews)

In [94]:
#pip install pyLDAvis

In [95]:
import warnings
warnings.filterwarnings('ignore')

import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig


In [96]:
# Latent Dirichlet Allocation Model
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [97]:
# Non-Negative Matrix Factorization Model
nmf = NMF(n_components=NUM_TOPICS)
data_nmf = nmf.fit_transform(data_vectorized) 

In [98]:
# Latent Semantic Indexing Model using Truncated SVD
lsi = TruncatedSVD(n_components=NUM_TOPICS)
data_lsi = lsi.fit_transform(data_vectorized)

In [99]:
# Get Topic Name
def get_topic_name(tok):
    topic_name_toks = []
    doc = nlp(" ".join(tok))
    pos = [token.pos_ for token in doc]
    
    def remove_at(j):
        topic_name_toks.append(tok[j].capitalize())
        pos.remove(pos[j])
        tok.remove(tok[j])
    for x in range(5):
        i = 0
        if x < 3:
            if  ("ADJ"   in pos) : i = pos.index("ADJ")
            elif("PROPN" in pos) : i = pos.index("PROPN")
            elif("NOUN"  in pos) : i = pos.index("NOUN")
            elif("ADV"   in pos) : i = pos.index("ADV")
            elif("VERB"  in pos) : i = pos.index("VERB")
            else :i=0
        elif x < 5:
            if  ("NOUN"  in pos) : i = pos.index("NOUN")
            elif("PROPN" in pos) : i = pos.index("PROPN")
            elif("VERB"  in pos) : i = pos.index("VERB")
            elif("ADV"   in pos) : i = pos.index("ADV")
            else :i=0
        
        remove_at(i)
    
    return " ".join(topic_name_toks) 

In [100]:
def get_simple_topic_name(tok, top_n=5):
    return " ".join(tok[:top_n])

In [101]:
# Functions for printing keywords for each topic
def get_selected_topics(model, vectorizer, top_n=10):
    topics = {}
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        name = get_simple_topic_name([vectorizer.get_feature_names()[i] for i in topic.argsort()[:-top_n - 1:-1]])
        print("Topic Name: "+name)
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
        topics[idx]=name
    return topics

In [102]:
# Keywords for topics clustered by Latent Dirichlet Allocation
print("LDA Model:")
selected_topics = get_selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
Topic Name: food customer owner love know
[('food', 10.442763406634644), ('customer', 8.666571815435159), ('owner', 6.25502609259244), ('love', 6.12835815102017), ('know', 6.109367830380814), ('talk', 5.886121293324908), ('lady', 5.418721762210301), ('chinese', 5.111554349453622), ('rudeness', 4.852374432324161), ('business', 4.588639665342643)]
Topic 1:
Topic Name: food tell ask table service
[('food', 56.91904027272681), ('tell', 44.91008688985316), ('ask', 41.407419247736385), ('table', 32.30837811390011), ('service', 26.565832260856297), ('wait', 25.910065817835868), ('waitress', 24.265851037637816), ('bring', 22.28567815979849), ('think', 21.838331956807423), ('eat', 20.79394128001787)]
Topic 2:
Topic Name: food service fry sauce chicken
[('food', 115.99735478583672), ('service', 75.93692461887088), ('fry', 75.7520475111909), ('sauce', 72.04436087157103), ('chicken', 69.43585902189652), ('love', 64.75217519934466), ('dish', 61.96386119846564), ('rice', 61.85617

In [103]:
selected_topics

{0: 'food customer owner love know', 1: 'food tell ask table service', 2: 'food service fry sauce chicken', 3: 'lunch menu seafood cod vegetable', 4: 'price friend location meal spot'}

In [155]:
# to be continued

In [ ]:

1> Clean the review -   machine readable (pre processing - remove stopwards, symbols, )
2> Aspects > Topics     Topic modelling  ()
3> Topics > catagories  catagorization   
4> Semetimants > rating score/Aspects > LARA
5> Optional > recommending


Between 17 - 23 Asad  - Not Available
                Karun - Not Available
                Ron   - Not Available
24 - 31         Asad  - step x 
                Dec 12 - Dec 16 (4 days)
      



In [6]:
def read_file(name,path):
    ret = None
    try:
        with open(path+"/"+name, 'r') as f:
            return f.read()
        
    except:
        with open(bucket+path+"/"+name) as f:
            return f.read()
    

In [7]:
x = read_file('stopwords.txt','config')

NameError: name 'bucket' is not defined